# Accuracy Analysis

In [95]:
import eikon as ek
import pandas as pd
import numpy as np
import datetime
import plotly
import plotly.express as px
import plotly.graph_objs as go
ek.set_app_key("f47c330480d74c598b7e8ebc2539424e91764dd8")

https://community.developers.refinitiv.com/questions/73493/get-eps-historical-data-for-stocks.html

### Accuracy Variables  

**TR.EPSActValue** - The company's actual value normalized to reflect the I/B/E/S default currency and corporate actions (e.g. stock splits). Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

**TR.EPSMean** - The statistical average of all broker estimates determined to be on the majority accounting basis. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

--> this is a analyst forecast variable

**TR.EPSActSurprise** - The difference between the actual and the last mean of the period, expressed as a percentage. Earnings Per Share is defined as the EPS that the contributing analyst considers to be that with which to value a security. This figure may include or exclude certain items depending on the contributing analyst's specific model.  

--> forecast error between actual EPS and TR.EPSMean  


### DataFrames

**df_accuracy** - basic dataframe containing quaterly data for all S&P 500 companies on EPS Actual, EPS Mean (the analyst forcast) and EPS Surprise (the forecast error in %)

**df_accuracy_new** - df_accuracy without extremely high or low values in the EPS Surprise column, so without outliers

**df_averages** - dataframe containing mean values of EPS Actual, EPS Mean and EPS Suprise over the entire time grouped by Instrument, takes df_accuracy_new as basis, so no outliers

**df_accuracy_yearly** - dataframe grouping quaterly datapoints into yearly data points

In [105]:
accuracy_variables = ['TR.EPSactValue.date', 'TR.EPSActValue', "TR.EPSMean", "TR.EPSActSurprise"]
df_accuracy, e = ek.get_data('0#.SPX',accuracy_variables, parameters = {'SDate':'0','EDate':'-40','Period':'FQ0','Frq':'FQ'})
df_accuracy["Date"] = pd.to_datetime(df_accuracy["Date"])
df_accuracy = df_accuracy.dropna()
df_accuracy

Instrument                      Date  Earnings Per Share - Actual  \
0        POOL.OQ 2023-02-16 07:00:00+00:00                         1.82   
1        POOL.OQ 2022-10-20 07:00:00+00:00                         4.78   
2        POOL.OQ 2022-07-21 07:00:00+00:00                         7.63   
3        POOL.OQ 2022-04-21 07:00:00+00:00                         4.23   
4        POOL.OQ 2022-02-17 07:00:00+00:00                         2.63   
...          ...                       ...                          ...   
20578      AVY.N 2014-01-31 08:30:00+00:00                         0.69   
20579      AVY.N 2013-10-25 08:30:00+00:00                         0.69   
20580      AVY.N 2013-07-23 08:30:00+00:00                         0.71   
20581      AVY.N 2013-04-24 08:30:00+00:00                         0.59   
20582      AVY.N 2013-01-30 07:35:00+00:00                         0.54   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                          1.987                                -8.405  
1                         4.5875                                 4.196  
2                          7.517                                 1.503  
3                        3.14867                                34.342  
4                          1.875                                40.267  
...                          ...                                   ...  
20578                       0.68                                 1.471  
20579                    0.63833                                 8.095  
20580                     0.7025                                 1.068  
20581                    0.57571                                 2.482  
20582                    0.48571                                11.178  

[19927 rows x 5 columns]

In [106]:
df_accuracy.dtypes

Instrument                                           string
Date                                    datetime64[ns, UTC]
Earnings Per Share - Actual                         Float64
Earnings Per Share - Mean                           Float64
Earnings Per Share - Actual Surprise                Float64
dtype: object

### Exploratory Data Analysis of Analyst Forcast Accuracy

### 1) Summary statistics for EPS Actual, EPS Mean (forecast), and EPS Surprise

In [107]:
df_accuracy.describe()

Earnings Per Share - Actual  Earnings Per Share - Mean  \
count                      19927.0                    19927.0   
mean                      1.381198                   1.287166   
std                       3.106833                   2.898835   
min                         -16.43                    -15.985   
25%                            0.5                     0.4625   
50%                           0.92                     0.8665   
75%                            1.6                    1.50094   
max                        133.441                  126.76571   

       Earnings Per Share - Actual Surprise  
count                               19927.0  
mean                             191.020225  
std                            23393.513652  
min                               -8858.503  
25%                                   0.468  
50%                                   4.548  
75%                                  12.259  
max                             3297926.087

Min and max values seem very high. Next step is to check for outliers and remove them for better results:  

**Removing Outliers:**

In [108]:
summary_stats = df_accuracy["Earnings Per Share - Actual Surprise"].describe()
Q1 = summary_stats.loc['25%']
Q3 = summary_stats.loc['75%']
IQR = Q3 - Q1
threshold = 7 #1.5 standard
surprise_outliers_removed = df_accuracy["Earnings Per Share - Actual Surprise"].loc[~((df_accuracy["Earnings Per Share - Actual Surprise"] < (Q1 - threshold * IQR)) | (df_accuracy["Earnings Per Share - Actual Surprise"] > (Q3 + threshold * IQR)))]
df_accuracy_new = df_accuracy.copy()
df_accuracy_new["Earnings Per Share - Actual Surprise"] = surprise_outliers_removed
df_accuracy_new

Instrument                      Date  Earnings Per Share - Actual  \
0        POOL.OQ 2023-02-16 07:00:00+00:00                         1.82   
1        POOL.OQ 2022-10-20 07:00:00+00:00                         4.78   
2        POOL.OQ 2022-07-21 07:00:00+00:00                         7.63   
3        POOL.OQ 2022-04-21 07:00:00+00:00                         4.23   
4        POOL.OQ 2022-02-17 07:00:00+00:00                         2.63   
...          ...                       ...                          ...   
20578      AVY.N 2014-01-31 08:30:00+00:00                         0.69   
20579      AVY.N 2013-10-25 08:30:00+00:00                         0.69   
20580      AVY.N 2013-07-23 08:30:00+00:00                         0.71   
20581      AVY.N 2013-04-24 08:30:00+00:00                         0.59   
20582      AVY.N 2013-01-30 07:35:00+00:00                         0.54   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                          1.987                                -8.405  
1                         4.5875                                 4.196  
2                          7.517                                 1.503  
3                        3.14867                                34.342  
4                          1.875                                40.267  
...                          ...                                   ...  
20578                       0.68                                 1.471  
20579                    0.63833                                 8.095  
20580                     0.7025                                 1.068  
20581                    0.57571                                 2.482  
20582                    0.48571                                11.178  

[19927 rows x 5 columns]

In [109]:
na_count = df_accuracy_new["Earnings Per Share - Actual Surprise"].isna().sum()
na_count

798

--> deteceted outliers at the given threshold

In [110]:
#removing NA
df_accuracy_new = df_accuracy_new.dropna()
df_accuracy_new

Instrument                      Date  Earnings Per Share - Actual  \
0        POOL.OQ 2023-02-16 07:00:00+00:00                         1.82   
1        POOL.OQ 2022-10-20 07:00:00+00:00                         4.78   
2        POOL.OQ 2022-07-21 07:00:00+00:00                         7.63   
3        POOL.OQ 2022-04-21 07:00:00+00:00                         4.23   
4        POOL.OQ 2022-02-17 07:00:00+00:00                         2.63   
...          ...                       ...                          ...   
20578      AVY.N 2014-01-31 08:30:00+00:00                         0.69   
20579      AVY.N 2013-10-25 08:30:00+00:00                         0.69   
20580      AVY.N 2013-07-23 08:30:00+00:00                         0.71   
20581      AVY.N 2013-04-24 08:30:00+00:00                         0.59   
20582      AVY.N 2013-01-30 07:35:00+00:00                         0.54   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                          1.987                                -8.405  
1                         4.5875                                 4.196  
2                          7.517                                 1.503  
3                        3.14867                                34.342  
4                          1.875                                40.267  
...                          ...                                   ...  
20578                       0.68                                 1.471  
20579                    0.63833                                 8.095  
20580                     0.7025                                 1.068  
20581                    0.57571                                 2.482  
20582                    0.48571                                11.178  

[19129 rows x 5 columns]

In [111]:
df_accuracy_new.describe()

Earnings Per Share - Actual  Earnings Per Share - Mean  \
count                      19129.0                    19129.0   
mean                      1.423885                   1.333085   
std                       3.151161                   2.948303   
min                         -16.43                    -15.985   
25%                           0.53                    0.50207   
50%                           0.95                    0.89964   
75%                           1.63                    1.53832   
max                        133.441                  126.76571   

       Earnings Per Share - Actual Surprise  
count                               19129.0  
mean                               6.929393  
std                               17.084754  
min                                  -81.69  
25%                                   0.506  
50%                                   4.376  
75%                                  11.482  
max                                  94.748

### 2) Forecast error distribution

**Surprise Distribution - with outliers**

In [119]:
fig = px.histogram(df_accuracy, x="Earnings Per Share - Actual Surprise", nbins=1000, title="EPS Surprise (%) Distribution (with outliers)")
fig.update_layout(yaxis=dict(tickformat=".2%"))
fig.show()

**Surprise Distribution - without outliers**

In [120]:
fig = px.histogram(df_accuracy_new, x="Earnings Per Share - Actual Surprise", nbins=1000, title="EPS Surprise (%) Distribution (outliers removed)")
fig.update_layout(yaxis=dict(tickformat=".2%"))
fig.show()

### 3) Mean EPS Actual, EPS Mean and EPS Surprise over all time periods per Instrument

In [122]:
df_averages = df_accuracy_new.groupby("Instrument").mean()
df_averages

/var/folders/tj/07dch2wn75v90cdvrd8w0xm80000gn/T/ipykernel_79969/3471284302.py:1: FutureWarning:

The default value of numeric_only in DataFrameGroupBy.mean is deprecated. In a future version, numeric_only will default to False. Either specify numeric_only or select only columns which should be valid for the function.



Earnings Per Share - Actual  Earnings Per Share - Mean  \
Instrument                                                           
A.N                             0.77122                   0.719961   
AAL.OQ                            0.257                   0.187633   
AAP.N                           1.98878                   1.966271   
AAPL.OQ                        0.792457                   0.736867   
ABBV.N                          1.87625                   1.824545   
...                                 ...                        ...   
YUM.N                           0.87561                    0.84134   
ZBH.N                            1.7895                   1.727077   
ZBRA.OQ                        2.389756                   2.252027   
ZION.OQ                         0.85825                   0.774196   
ZTS.N                          0.718049                   0.663742   

            Earnings Per Share - Actual Surprise  
Instrument                                        
A.N                                     7.255585  
AAL.OQ                                  6.460925  
AAP.N                                   1.769024  
AAPL.OQ                                 7.065512  
ABBV.N                                   3.27955  
...                                          ...  
YUM.N                                      5.429  
ZBH.N                                     4.5081  
ZBRA.OQ                                 5.404537  
ZION.OQ                                 9.387575  
ZTS.N                                    7.38978  

[502 rows x 3 columns]

Summary statistics per instrument

In [123]:
df_averages.describe()

Earnings Per Share - Actual  Earnings Per Share - Mean  \
count                        502.0                      502.0   
mean                      1.380964                   1.292809   
std                       2.420182                   2.275438   
min                      -0.291842                  -0.307703   
25%                       0.602805                   0.577975   
50%                       1.012051                   0.954061   
75%                       1.589085                   1.492998   
max                       46.97222                  44.416074   

       Earnings Per Share - Actual Surprise  
count                                 502.0  
mean                               6.989945  
std                                6.364702  
min                                 -54.155  
25%                                3.882537  
50%                                 6.01172  
75%                                9.504716  
max                               48.632652

### 4) Number of Instruments per surprise percentile group for each year

In [125]:
# grouping data by year
df_accuracy_yearly, e = ek.get_data('0#.SPX', accuracy_variables, parameters = {'SDate':'0','EDate':'-40','Period':'FY0','Frq':'FY'})
df_accuracy_yearly["Date"] = pd.to_datetime(df_accuracy_yearly["Date"]).dt.year
df_accuracy_yearly = df_accuracy_yearly.dropna()

In [126]:
bins = [-100, -50, -20, -10, -5, 0, 5, 10, 20, 30, 40, 50, 60, 80, 100]

# Group the data by year and calculate the percentile counts for each year
df_percentiles = pd.DataFrame(index=range(df_accuracy_yearly["Date"].min(), df_accuracy_yearly["Date"].max()+1),
                              columns=[f"{bins[i]}-{bins[i+1]}" for i in range(len(bins)-1)])
for year in df_percentiles.index:
    df_year = df_accuracy_yearly[df_accuracy_yearly["Date"] == year]
    percentile_counts = pd.cut(df_year["Earnings Per Share - Actual Surprise"], bins=bins, labels=df_percentiles.columns).value_counts().sort_index()
    df_percentiles.loc[year] = percentile_counts.values

df_percentiles

-100--50 -50--20 -20--10 -10--5 -5-0  0-5 5-10 10-20 20-30 30-40 40-50  \
1983        0       4       3      3    9    4    2     0     0     0     0   
1984        5      10      16     16   51   45   21     9     4     3     2   
1985        3      18       8     13   51   56   16     9     1     4     0   
1986        7      13      14     21   61   51    8     5     0     2     1   
1987        8      19      22     24   26   37   15    12     4     1     0   
1988        6      17      12     15   46   59    9    11     2     1     1   
1989        5       8       7     14   44   77   15     9     1     3     1   
1990        8      10      10     19   55   60   19     8     1     1     0   
1991       13      25      25     28   93   87   28    19     2     2     0   
1992        8      16      17     20   72   82   28     6     5     1     0   
1993        6      11      10     19   82   97   20    14     6     1     2   
1994        2       9       8     10   78  117   30    10     6     5     2   
1995        0       5       6     10   70  150   24    10     4     1     0   
1996        2       1       5     11   78  149   28    11     5     1     3   
1997        2       1       6      7   76  169   24    10     1     1     0   
1998        3      10       2     12   76  142   42    13     6     1     0   
1999        2       8       7     12   77  170   17    10     5     1     2   
2000        5       3       5     11   75  174   26    11     9     1     1   
2001        8       8       7      8   83  175   28    15     2     0     2   
2002        2       3       8      8   99  201   20    11     4     1     3   
2003        4       4       3     12   89  215   25    14     4     4     1   
2004        0       4       7      6   82  222   36    10     6     1     1   
2005        1       0       9     12   93  215   29    12     8     0     2   
2006        2      12       5      5   91  227   33    12     4     1     4   
2007        1       3       6      5   96  246   31    12     3     0     1   
2008        4       6       5     14  103  232   26    11     1     3     0   
2009        6      14      17     17   91  201   38    13     8     2     0   
2010        0       4      10      8   85  213   54    24     9     5     3   
2011        2       5      10      7   94  252   39    18     4     0     3   
2012        2       4       8     15  111  245   31    17     5     1     0   
2013        1       4       5      8  102  268   35    12     7     4     0   
2014        1       4       7      6  125  278   23    10     5     2     1   
2015        3       5       5     10  126  274   26    11     8     1     2   
2016        2       8       2     17   99  281   32    22     4     2     0   
2017        4       8       6     11  101  293   34    13     2     3     2   
2018        0       4      11      8   97  309   33    12     8     1     0   
2019        1       6      10      7  122  298   26    14     1     1     2   
2020        2       6       4      9   98  300   35    22     6     4     3   
2021        5       9      10     12   72  258   65    40    11     5     2   
2022        0       7       8     13   98  316   30    11     4     3     2   
2023        1       5       7     16   81  195   20    10     3     3     0   

     50-60 60-80 80-100  
1983     0     0      0  
1984     1     1      3  
1985     0     2      0  
1986     1     0      5  
1987     3     0      3  
1988     0     0      0  
1989     2     0      1  
1990     1     0      0  
1991     2     3      8  
1992     2     0      3  
1993     0     1      2  
1994     0     0      2  
1995     2     1      0  
1996     0     1      0  
1997     0     0      1  
1998     1     1      1  
1999     1     0      0  
2000     0     0      2  
2001     2     1      0  
2002     0     0      0  
2003     3     2      0  
2004     1     2      1  
2005     1     1      0  
2006     1     0      0  
2007     0     0      0  
2008     1

### 5) Average forecast error (surprise) over the last 10 years

In [127]:
df_accuracy_yearly

Instrument  Date  Earnings Per Share - Actual  \
0        POOL.OQ  2023                         18.7   
1        POOL.OQ  2022                        15.92   
2        POOL.OQ  2021                         9.13   
3        POOL.OQ  2020                          6.4   
4        POOL.OQ  2019                         5.62   
...          ...   ...                          ...   
20609      AVY.N  1996                         1.34   
20610      AVY.N  1995                        0.985   
20611      AVY.N  1994                         0.72   
20612      AVY.N  1993                        0.665   
20613      AVY.N  1992                         0.51   

       Earnings Per Share - Mean  Earnings Per Share - Actual Surprise  
0                          18.78                                -0.426  
1                          15.19                                 4.806  
2                           8.45                                 8.047  
3                        6.34111                                 0.929  
4                           5.65                                -0.531  
...                          ...                                   ...  
20609                    1.32889                                 0.836  
20610                     0.9575                                 2.872  
20611                    0.72389                                -0.537  
20612                    0.69063                                -3.711  
20613                    0.53773                                -5.157  

[14471 rows x 5 columns]

In [129]:
# Group the data by year and calculate the mean error for each year
df_yearly_mean = df_accuracy_yearly.groupby("Date")["Earnings Per Share - Actual Surprise"].mean().reset_index()

# Create a line plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_yearly_mean ["Date"], y=df_yearly_mean ["Earnings Per Share - Actual Surprise"], mode="lines", name="Average S&P500 Forecast Error"))

# Set the title and axis labels
fig.update_layout(title="Yearly Average Forecast Error of S&P 500 Companies",
                   xaxis_title="Year",
                   yaxis_title="Forecast Error (%)")

# Display the plot
fig.show()

In [104]:
# Group the data by year and calculate the mean error for each year
df_quaterly_mean = df_accuracy.groupby("Date")["Earnings Per Share - Actual Surprise"].mean().reset_index()

# Create a line plot using Plotly
fig = go.Figure()
fig.add_trace(go.Scatter(x=df_accuracy ["Date"], y=df_accuracy ["Earnings Per Share - Actual Surprise"], mode="lines", name="Average S&P500 Forecast Error"))

# Set the title and axis labels
fig.update_layout(title="Quaterly Average Forecast Error of S&P 500 Companies",
                   xaxis_title="Quater",
                   yaxis_title="Forecast Error (%)")

# Display the plot
fig.show()